In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


words = open('names.txt', 'r').read().splitlines()


chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

print(f"Vocabulary size: {vocab_size}")


block_size = 3 # Context length: how many characters do we take to predict the next one?


def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

# Split data into training (80%), validation (10%), and test (10%)
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


g = torch.Generator().manual_seed(2147483647)

n_embd = 10
n_hidden = 200

#
C = torch.randn((vocab_size, n_embd), generator=g)


W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.01


W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)


W_direct = torch.randn((n_embd * block_size, vocab_size), generator=g) * 0.01


parameters = [C, W1, b1, W2, b2, W_direct]

print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

for p in parameters:
    p.requires_grad = True



max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):


    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]


    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)


    h = torch.tanh(x @ W1 + b1)
    logits_hidden = h @ W2 + b2


    logits_direct = x @ W_direct


    logits = logits_hidden + logits_direct


    loss = F.cross_entropy(logits, Yb)


    for p in parameters:
        p.grad = None
    loss.backward()


    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad





@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]

    emb = C[x]
    x_flat = emb.view(emb.shape[0], -1)

    h = torch.tanh(x_flat @ W1 + b1)
    logits_hidden = h @ W2 + b2
    logits_direct = x_flat @ W_direct
    logits = logits_hidden + logits_direct

    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

print("\nFinal Loss:")
split_loss('train')
split_loss('val')



print("\nGenerated Names:")
g_sample = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:

        emb = C[torch.tensor([context])]
        x_flat = emb.view(1, -1)

        h = torch.tanh(x_flat @ W1 + b1)
        logits_hidden = h @ W2 + b2
        logits_direct = x_flat @ W_direct
        logits = logits_hidden + logits_direct

        probs = F.softmax(logits, dim=1)

        ix = torch.multinomial(probs, num_samples=1, generator=g_sample).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out[:-1]))


Vocabulary size: 27
torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])
Total parameters: 12707

Final Loss:
train 2.0408642292022705
val 2.1058578491210938

Generated Names:
carlaizabelle
khyimrix
tatyanna
sancer
hubefa
esara
kaeli
nellara
chaiir
kaleigh
ham
join
quint
salin
alian
quinterri
jaryxin
kaellissabee
deciia
gian
